In [1]:
import torch
from slim_gsgp.main_mo_gp import mo_gp
from slim_gsgp.datasets.data_loader import load_ppb
from slim_gsgp.utils.utils import train_test_split

# General Multi-Objective Settings
MO_FFUNCS = ["rmse", "size"]
MO_MIN_FLAGS = [True, True]
MO_TS_SIZES = [4, 2]
IDEAL_POINTS = [0.0, 0.0]


X, y = load_ppb(X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4, seed=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5, seed=42)

# Helper function to run and print
def run_scenario(name, **kwargs):
    print(f"\n{'='*60}\nRunning Scenario: {name}\n{'='*60}")
    default_args = {
        'pop_size': 50,
        'n_iter': 5,
        'n_elites': 1,
        'ideal_candidate_values': None
    }
    params = {**default_args, **kwargs}
    
    mo_gp(
        X_train=X_train, y_train=y_train,
        X_test=X_val, y_test=y_val,
        dataset_name='ppb_test',
        fitness_functions=MO_FFUNCS,
        minimization_flags=MO_MIN_FLAGS,
        tournament_sizes=MO_TS_SIZES,
        verbose=1,
        log_level=0,
        n_jobs=2,
        **params
    )

# =============================================================================
# 1. NSGA-II Puro
# =============================================================================
run_scenario(
    "1. NSGA-II Puro",
    selector_strategy="nsga2",
    survival_strategy="nsga2",
    n_elites=1
)

# =============================================================================
# 2. Nested Tournament + NSGA-II Survival
# =============================================================================
run_scenario(
    "2. Nested Parent Selection + NSGA-II Survival",
    selector_strategy="nested_tournament", # Seleção de Pais Hierárquica
    survival_strategy="nsga2"              # Sobrevivência Competitiva
)

# =============================================================================
# 3. Nested Tournament + Direct Replacement (No Elitism)
# =============================================================================
run_scenario(
    "3. Nested + Direct Replacement (No Elitism)",
    selector_strategy="nested_tournament",
    survival_strategy="generational",
    n_elites=0
)

# =============================================================================
# 4. Nested Tournament + Standard Elitism (Rank/CD)
# =============================================================================
run_scenario(
    "4. Nested + Direct Replacement + Rank Elitism",
    selector_strategy="nested_tournament",
    survival_strategy="generational",
    n_elites=2,
    ideal_candidate_values=None
)

# =============================================================================
# 5. Nested Tournament + Ideal Candidate Elitism
# =============================================================================
run_scenario(
    "5. Nested + Direct Replacement + Ideal Candidate Elitism",
    selector_strategy="nested_tournament",
    survival_strategy="generational",
    n_elites=2,
    ideal_candidate_values=IDEAL_POINTS
)


Running Scenario: 1. NSGA-II Puro

------------------------------------------------------------------------------------------------------------------------------
| Dataset         |  Gen   | Train Fitness           | Test Fitness            | Timing (s) |  Nodes   |
------------------------------------------------------------------------------------------------------------------------------
| ppb_test        |      0 | 4929979.5000 | -1124874.1250 | 5411280.5000 | -3923.4021 |     7.0007 |       63 |
------------------------------------------------------------------------------------------------------------------------------
| ppb_test        |      1 | 5616696.0000 | -1455569.1250 | 5805283.5000 | -6055.9888 |     0.5604 |       63 |
------------------------------------------------------------------------------------------------------------------------------
| ppb_test        |      2 | 5616696.0000 | -1455569.1250 | 5805283.5000 | -6055.9888 |     0.5824 |       63 |
---------------

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [2]:
import pytest
pytest.main(["tests/test_mo_gp.py"])
pytest.main(["tests/test_validate_inputs.py"])
pytest.main(["tests/test_utils.py"])

============================= test session starts =============================
platform win32 -- Python 3.10.16, pytest-8.3.4, pluggy-1.5.0
rootdir: c:\Users\filip\OneDrive\Documentos\NOVA\Tese\slim-main
plugins: anyio-4.6.2, mock-3.15.1
collected 28 items

tests\test_mo_gp.py ...........FF...F.F........F                         [100%]

================================== FAILURES ===================================
____________ test_mo_gp_elitism_calculates_correct_offspring_count ____________

mock_evolve = <MagicMock name='evolve_population' id='1588604455520'>

    @patch('slim_gsgp.algorithms.GP.mogp.MOGP.evolve_population')
    def test_mo_gp_elitism_calculates_correct_offspring_count(mock_evolve):
        mock_pop = MockMOPopulation([])
        mock_pop.population = []
        mock_evolve.return_value = (mock_pop, 0.0)
        pop_size = 20
        n_elites = 2
    
        #execute mo_gp with elitism=True and n_elites=2
>       mo_gp(
            X_train=valid_X_train,
        

<ExitCode.TESTS_FAILED: 1>

In [ ]:
# !pip install pytest-mock
# !pip install pytest